In [107]:
#Importing data
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
import warnings
warnings.filterwarnings('ignore')

#Importing data
metadata = pd.read_csv('metadata_labeled.csv')
metadata = metadata.drop_duplicates(subset='Run', keep='first')
metadata = metadata.drop('Label',axis=1)
multiqc = pd.read_csv('multiqc_data.csv')
label= pd.read_csv('labels.csv',sep='\t')
#Merge metadata and execution dataframe on Run
df = pd.merge(metadata, multiqc, on='Run', how='inner')
df = pd.merge(df, label, on='Run', how='inner')
df.index = df['Run']
#Removing columns with no data
df = df.dropna(axis=1, how='all')
#Removing columns with only one unique value
df = df.loc[:,df.apply(pd.Series.nunique) != 1]
#Removing columns with more than 40% missing values
df = df.loc[:, df.isnull().mean() < .4]
#Removing columns with more than 40% zeros
df = df.loc[:, (df == 0).mean() < .4]
#Remove non-numeric columns
df = df.select_dtypes(include=[np.number])
df = df.loc[:,~df.columns.duplicated()]
#shuffle df
df = df.sample(frac=1)
# remove 'None_mqc-generalstats-fastqc-percent_duplicates', 'None_mqc-generalstats-fastqc-percent_gc', 'None_mqc-generalstats-fastqc-avg_sequence_length', 'None_mqc-generalstats-fastqc-median_sequence_length', 'None_mqc-generalstats-fastqc-percent_fails', 'None_mqc-generalstats-fastqc-total_sequences coluumns from df
df = df.drop(['None_mqc-generalstats-fastqc-percent_duplicates', 'None_mqc-generalstats-fastqc-percent_gc', 'None_mqc-generalstats-fastqc-avg_sequence_length', 'None_mqc-generalstats-fastqc-median_sequence_length', 'None_mqc-generalstats-fastqc-percent_fails', 'None_mqc-generalstats-fastqc-total_sequences'], axis=1)
X = df.iloc[:,0:-5] #features
y = df.iloc[:,-5:] #labels
sc=StandardScaler()
X = sc.fit_transform(X)
X_train = X[0:80]; X_test = X[80:]
y_train = y[0:80]; y_test = y[80:]
del label, metadata, multiqc, sc
y_test.index

Index(['SRR794398', 'ERR020241', 'ERR022393', 'ERR062924', 'ERR018442',
       'ERR018550', 'ERR018454', 'ERR018448', 'ERR018463', 'ERR018436',
       'ERR023204', 'ERR022463', 'ERR016316', 'ERR018197', 'ERR018423',
       'ERR018491', 'ERR062953', 'ERR062973'],
      dtype='object', name='Run')

In [108]:
# # Test for other samples here
# metadata_test = pd.read_csv('metadata_test.csv'); multiqc_test = pd.read_csv('multiqc_testdata.csv')
# # metadata_test = metadata_test.drop_duplicates(subset='Run', keep='first')
# df_test = pd.merge(metadata_test, multiqc_test, on='Run', how='inner'); df_test.index = df_test['Run']
# df = df.drop(['M1', 'M2', 'M3', 'M4', 'M5'], axis=1)
# features = df.columns; 
# df_test = df_test[features]
# sc=StandardScaler()
# X_test2 = sc.fit_transform(df_test); del df_test, metadata_test, multiqc_test, sc

In [114]:
# Train the model with M1 machine
machine_name = 'M5'   # Enter your machine name here
y_train_machine = y_train[machine_name]
y_test_machine = y_test[machine_name]

# Initialize and train the model
model = RandomForestRegressor()
model.fit(X_train, y_train_machine)

# Predict the values
y_pred = model.predict(X_test)

# Create a DataFrame for the actual and predicted values
df_res = pd.DataFrame({'Actual': y_test_machine, 'Predicted': y_pred})

# Save the DataFrame to a CSV file
df_res.to_csv('actual_vs_predicted.csv', index=True)

# Calculate and print evaluation metrics
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test_machine, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test_machine, y_pred))
print('R2 Score:', metrics.r2_score(y_test_machine, y_pred))
print('Correlation:', df_res['Actual'].corr(df_res['Predicted']))
print("actual_vs_predicted.csv file is created in the current working directory")

# Display the first few rows of the DataFrame
df_res

Mean Absolute Error: 86.83555555555557
Mean Squared Error: 17615.029533333334
R2 Score: 0.9377097162915822
Correlation: 0.9711747166935497
actual_vs_predicted.csv file is created in the current working directory


,Actual,Predicted
Run,,
SRR794398,1118,1112.59
ERR020241,2152,2170.04
ERR022393,714,765.32
ERR062924,448,488.55
ERR018442,1250,1276.42
ERR018550,649,646.16
ERR018454,1405,1225.05
ERR018448,659,736.24
ERR018463,728,836.00


In [110]:
# # Predict for new test data
# y_pred2 = model.predict(X_test2)
# pd.DataFrame(y_pred2).to_csv('predicted_testdata.csv', index=False)